<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Model IO Exercise 

The purpose of this exercise is to test your understanding of building out Model IO systems. You will also hopefully notice the need to chain responses together, which we will cover later in this course!

Watch the video for a full overview on minimum outputs this class should be capable of, but feel free to expand on this project, or to just treat it as a code-along!

## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

### Suggested Imports

Feel free to accomplish this task however you prefer!

In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

In [2]:
import os
open_ai_api_key = os.getenv('OPENAI_API_KEY')

In [25]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        system_prompt = SystemMessagePromptTemplate.from_template("You are an history teacher and your job is to come up with tough\
        historical question about the {topic} that has a date as the correct answer. The question should be single line question")

        human_prompt = HumanMessagePromptTemplate.from_template("Create a question on the {topic}.")
        
        chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])
        
        request = chat_prompt.format_prompt(topic=topic).to_messages()

        model = ChatOpenAI(api_key=open_ai_api_key)
        
        question = model(request).content

        return question
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        system_prompt = SystemMessagePromptTemplate.from_template("You are a robot which gives answer about history as dates only")

        output_parser = DatetimeOutputParser()
        human_prompt = HumanMessagePromptTemplate.from_template("Answer the following question: {question} \n {output_instructions}")
        
        chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])
        
        request = chat_prompt.format_prompt(question=question, output_instructions=output_parser.get_format_instructions()).to_messages()

        model = ChatOpenAI(api_key=open_ai_api_key)
        
        answer = model(request).content

        try:
            correct_datetime=output_parser.parse(answer)
        except:
            model = OpenAI(api_key=open_ai_api_key)
            new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=model)
            correct_datetime
        
        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        

        
        return user_datetime
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        pass
        

### Example Usage

Feel free to expand or edit this project. To keep things simple we have every method return an object that will then feed into a new method!

In [26]:
quiz_bot = HistoryQuiz()

In [22]:
question = quiz_bot.create_history_question(topic='World War 2')

In [23]:
question

'On what date did the attack on Pearl Harbor occur?'

In [27]:
ai_answer = quiz_bot.get_AI_answer(question)

OutputParserException: Could not parse datetime string: The attack on Pearl Harbor occurred on December 7, 1941. The corresponding datetime string would be "1941-12-07T00:00:00.000000Z".

In [ ]:
# Day After Pearl Harbor
ai_answer

In [40]:
user_answer = quiz_bot.get_user_answer(question)

On what date did the United States officially enter World War II?


Enter the year:  1941
Enter the month (1-12):  12
Enter the day (1-31):  1


In [41]:
user_answer

datetime.datetime(1941, 12, 1, 0, 0)

In [42]:
quiz_bot.check_user_answer(user_answer,ai_answer)

The difference between the dates is: -7 days, 0:00:00
